In [ ]:
import torch
import pickle

from pathlib import Path
from itertools import combinations
from matplotlib.colors import LogNorm

import numpy as np
import matplotlib.pyplot as plt
import mdtraj as md
import nglview as nv

blue = (70 / 255, 110 / 255, 250 / 255)

# Plumed

In [155]:
molecule = "1fme"

## Visualization

In [156]:
gro_path = f"./{molecule.upper()}/folded.gro"
gro_traj = md.load(gro_path)
print(gro_traj)

gro_traj.center_coordinates()
gro_view = nv.show_mdtraj(gro_traj)
gro_view.add_representation("licorice", selection="protein")

gro_ca_atoms = [atom for atom in gro_traj.topology.atoms if atom.name == "CA"]
gro_ca_indices = [atom.index + 1 for atom in gro_ca_atoms]
print("Alpha carbon indices:", gro_ca_indices)

gro_view

<mdtraj.Trajectory with 1 frames, 504 atoms, 28 residues, and unitcells>
Alpha carbon indices: [5, 20, 37, 58, 72, 82, 104, 125, 147, 154, 178, 192, 212, 236, 250, 265, 287, 302, 321, 345, 357, 377, 396, 411, 433, 453, 475, 482]


NGLWidget()

In [157]:
pdb_path = f"./{molecule.upper()}/folded.pdb"
pdb_traj = md.load_pdb(pdb_path)
print(pdb_traj)

pdb_traj.center_coordinates()
pdb_view = nv.show_mdtraj(pdb_traj)
pdb_view.add_representation("licorice", selection="protein")

pdb_ca_atoms = [atom for atom in pdb_traj.topology.atoms if atom.name == "CA"]
pdb_ca_indices = [atom.index + 1 for atom in pdb_ca_atoms]
print("Alpha carbon indices:", pdb_ca_indices)

pdb_view

<mdtraj.Trajectory with 1 frames, 504 atoms, 28 residues, and unitcells>
Alpha carbon indices: [2, 19, 36, 57, 71, 81, 103, 124, 146, 153, 177, 191, 211, 235, 249, 264, 286, 301, 320, 344, 356, 376, 395, 410, 432, 452, 474, 481]


NGLWidget()

In [158]:
np.array(gro_ca_indices) == np.array(pdb_ca_indices)

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])

In [159]:
cnt = 0
for i in range(len(gro_ca_indices)):
    for j in range(i+1, len(gro_ca_indices)):
        print(f"d{cnt+1}: DISTANCE ATOMS={gro_ca_indices[i]},{gro_ca_indices[j]}")
        cnt += 1
        
print("\n")
print("ARG=" + ",".join(f"d{i+1}" for i in range(cnt)))

d1: DISTANCE ATOMS=5,20
d2: DISTANCE ATOMS=5,37
d3: DISTANCE ATOMS=5,58
d4: DISTANCE ATOMS=5,72
d5: DISTANCE ATOMS=5,82
d6: DISTANCE ATOMS=5,104
d7: DISTANCE ATOMS=5,125
d8: DISTANCE ATOMS=5,147
d9: DISTANCE ATOMS=5,154
d10: DISTANCE ATOMS=5,178
d11: DISTANCE ATOMS=5,192
d12: DISTANCE ATOMS=5,212
d13: DISTANCE ATOMS=5,236
d14: DISTANCE ATOMS=5,250
d15: DISTANCE ATOMS=5,265
d16: DISTANCE ATOMS=5,287
d17: DISTANCE ATOMS=5,302
d18: DISTANCE ATOMS=5,321
d19: DISTANCE ATOMS=5,345
d20: DISTANCE ATOMS=5,357
d21: DISTANCE ATOMS=5,377
d22: DISTANCE ATOMS=5,396
d23: DISTANCE ATOMS=5,411
d24: DISTANCE ATOMS=5,433
d25: DISTANCE ATOMS=5,453
d26: DISTANCE ATOMS=5,475
d27: DISTANCE ATOMS=5,482
d28: DISTANCE ATOMS=20,37
d29: DISTANCE ATOMS=20,58
d30: DISTANCE ATOMS=20,72
d31: DISTANCE ATOMS=20,82
d32: DISTANCE ATOMS=20,104
d33: DISTANCE ATOMS=20,125
d34: DISTANCE ATOMS=20,147
d35: DISTANCE ATOMS=20,154
d36: DISTANCE ATOMS=20,178
d37: DISTANCE ATOMS=20,192
d38: DISTANCE ATOMS=20,212
d39: DISTANCE ATOMS

# TICA coordinates & trjconv

In [ ]:
xtc_path = "/bioemu/opes/simulations/gtt/tica/debug/0.xtc"
analysis_dir = Path(f"/bioemu/opes/simulations/gtt/tica/debug/")
tpr_path = "/bioemu/opes/data/GTT/nvt_0.tpr"
# cmd = f"gmx trjconv -f {xtc_path} -o {analysis_dir}/0_debug.xtc -pbc nojump "
cmd = f"gmx trjconv -f {xtc_path} -s {tpr_path} -pbc nojump -o {analysis_dir}/0.xtc"
print(cmd)

gmx trjconv -f /bioemu/opes/simulations/gtt/tica/debug/0.xtc -s /bioemu/opes/data/GT/nvt_0.tpr -pbc nojump -o /bioemu/opes/simulations/gtt/tica/debug/0.xtc


In [ ]:
method="tica"
moelcule="1fme"
date="0908_152748"

print(f"python analysis_opes.py --config-name {method}-{molecule} date={date} analysis.skip_steps=0 analysis.unit_steps=1 +tags=['vast','debug']")

python analysis_opes.py --config-name tica-1fme date=0908_152748 analysis.skip_steps=0 analysis.unit_steps=1 +tags=['vast','debug']


# OPES results

In [6]:
xtc_path = "/home/shpark/prj-mlcv/lib/bioemu/opes/simulations/gtt/tica/debug/0.xtc"
gro_path = "/home/shpark/prj-mlcv/lib/bioemu/opes/data/GTT/folded.gro"
traj = md.load(xtc_path, top=gro_path)
view = nv.show_mdtraj(traj)
view.add_representation("licorice", selection="protein")
view.display()

OSError: No such file: /home/shpark/prj-mlcv/lib/bioemu/opes/simulations/gtt/tica/debug/0.xtc

In [ ]:
export GMX_CUDA_GRAPH=0
export CUDA_LAUNCH_BLOCKING=1
gmx mdrun -s md.tpr -deffnm md -ntmpi 1 -ntomp 1 -reprod -pme cpu
gmx mdrun -s ./data/1FME/nvt_0.tpr -deffnm simulations/1fme/tica/debug/0 -plumed simulations/1fme/tica/debug/0/plumed.dat -nsteps 1000000 -reseed 0 -ntomp 1 -bonded gpu -nb gpu -pme gpu -dlb no -reprod -pme cpu

In [ ]:
bioemu/opes/simulations/1fme/tica/crash/2.edr

In [ ]:
gmx trjconv -s nvt_0.tpr -f /bioemu/opes/simulations/1fme/tica/debug/0.xtc -dump 0 -center -pbc mol -ur compact -o start_center.gro

In [92]:
gro_path = "/home/shpark/prj-mlcv/lib/bioemu/opes/data/1FME/boxed.gro"
traj = md.load(gro_path)
view = nv.show_mdtraj(traj)
view.add_representation("licorice", selection="protein")
view.display()

NGLWidget()

In [ ]:
# gmx grompp -f 1fme.mdp -c start_center.gro -p topol.top -o nvt_1.tpr -maxwarn 2
# gmx grompp -f 1fme_2.mdp -c start_center.gro -p topol.top -o nvt_2.tpr -maxwarn 2

In [ ]:
gmx editconf -f folded.gro -o boxed.gro -c -d 1.0 -bt cubic
gmx solvate -cp boxed.gro -cs spc216.gro -o solv.gro -p topol.top
gmx grompp -f ions.mdp -c solv.gro -p topol.top -o ions.tpr -maxwarn 1
gmx genion -s ions.tpr -o solv_ions.gro -p topol.top -pname NA -nname CL -neutral -conc 0.065
13
gmx grompp -f 1fme_ions.mdp -c solv_ions.gro -p topol.top -o nvt_ion.tpr

In [ ]:
gmx grompp -f ions.mdp -c solv.gro -p topol.top -o ions.tpr
gmx genion -s ions.tpr -o solv_ions.gro -p topol.top -nname CL -neutral

In [ ]:
# GTT

gmx editconf -f folded.gro -o boxed.gro -bt cubic -box 5.2 5.2 5.2 -c

gmx solvate -cp boxed.gro -cs spc216.gro -o solv.gro -p topol.top

gmx grompp -f ions.mdp -c solv.gro -p topol.top -o ions.tpr -maxwarn 1

gmx genion -s ions.tpr -o solv_ions.gro -p topol.top -nname CL -neutral

gmx grompp -f em.mdp -c solv_ions.gro -p topol.top -o em.tpr

gmx mdrun  -deffnm em -v

gmx grompp -f gtt.mdp -c em.gro -p topol.top -o nvt_1.tpr

In [ ]:
# 1FME
gmx pdb2gmx -f folded.pdb -o folded.gro -p topol.top -ff charmm27 -water tip3p --ignh

gmx grompp -f em.mdp -c solv_ions.gro -p topol.top -o em.tpr -maxwarn 1

gmx mdrun -deffnm em -ntmpi 1 -nb gpu